In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
from fastapi import FastAPI, HTTPException, Security, Depends, UploadFile, File
from fastapi.security import APIKeyHeader
from pydantic import BaseModel
import torch
import timm
from torchvision import transforms
from PIL import Image
import io

app = FastAPI(
    title="Deepfake Detector API",
    description="API for classifying images as real or fake",
    version="1.0.0"
)

# Configuration
MODEL_PATH = r"C:\Users\ZEBA FATHIMA\Downloads\deepfake\deepfake_detector_best.pth"
IMG_SIZE = 224
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model not found at {MODEL_PATH}")
checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=2)
model.load_state_dict(checkpoint["model_state"])
model.to(DEVICE)
model.eval()
classes = checkpoint.get("classes", ["real", "fake"])

# Image transformation
tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# API key for authentication
API_KEY = "your-secret-api-key"
api_key_header = APIKeyHeader(name="X-API-Key")

# Dependency to validate API key
async def verify_api_key(api_key: str = Depends(api_key_header)):
    if api_key != API_KEY:
        raise HTTPException(status_code=401, detail="Invalid API key")
    return api_key

# Pydantic model for response
class PredictionResponse(BaseModel):
    prediction: str
    confidence: float

@app.post("/predict", response_model=PredictionResponse)
async def predict(file: UploadFile = File(...), api_key: str = Depends(verify_api_key)):
    """
    Classify an image as real or fake.
    """
    try:
        # Read and process the image
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("RGB")
        input_tensor = tfms(image).unsqueeze(0).to(DEVICE)

        # Make prediction
        with torch.no_grad():
            outputs = model(input_tensor)
            prob = torch.nn.Softmax(dim=1)(outputs)
            confidence, pred_idx = torch.max(prob, 1)
            label = classes[pred_idx.item()]

        return {
            "prediction": label,
            "confidence": float(confidence.item())
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")

@app.get("/")
async def root():
    return {"message": "Welcome to the Deepfake Detector API. Use /predict to classify images."}

In [ ]:
import streamlit as st
import requests
from PIL import Image
import io

st.set_page_config(page_title="Deepfake Detector", layout="centered")

st.title("🖼️ Deepfake Detector")
st.write("Upload an image to classify as Real or Fake.")

# FastAPI endpoint URL
API_URL = "http://localhost:8000/predict"
API_KEY = "your-secret-api-key"  # Replace with the same key as in api.py

uploaded_file = st.file_uploader("Choose an image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Convert image to bytes
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format="PNG")
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Make request to FastAPI backend
        headers = {"X-API-Key": API_KEY}
        files = {"file": ("image.png", img_byte_arr, "image/png")}
        response = requests.post(API_URL, files=files, headers=headers)

        if response.status_code == 200:
            result = response.json()
            prediction = result["prediction"]
            confidence = result["confidence"]

            st.markdown(f"**Prediction:** {prediction}")
            st.markdown(f"**Confidence:** {confidence*100:.2f}%")
        else:
            st.error(f"❌ API error: {response.json().get('detail', 'Unknown error')}")

    except requests.exceptions.RequestException as e:
        st.error(f"❌ Failed to connect to the API: {str(e)}. Ensure the FastAPI server is running.")

st.markdown("---")
st.caption("Connected to FastAPI backend at: " + API_URL)

In [ ]:
uvicorn api:app --host 0.0.0.0 --port 8000

In [ ]:
curl -X POST "http://localhost:8000/predict" -H "X-API-Key: your-secret-api-key" -F "file=@path/to/image.jpg"

In [ ]:
streamlit run streamlit_app.py